In [1]:
%load_ext autoreload
%autoreload 2

here the objective is to fit the logit graph model to multiple different graph domains and then visualize the original vs fitted graphs

In [2]:

import sys
import os
sys.path.append('../')

#Graph imports
import src.graph as graph
import src.logit_estimator as estimator
import src.utils as utils
import src.model_selection as model_selection
import src.gic as gic
import src.param_estimator as pe
import src.graph as graph
import src.model_selection as ms

# usual imports
import matplotlib.pyplot as plt
import pickle
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import random
import networkx as nx
from numpy import errstate

from IPython.display import display
from pyvis.network import Network
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [4]:
FOLDER_OUTPUT = f'runs/fitted_graphs_comparison_facebook5'
out_files = os.listdir(FOLDER_OUTPUT)
out_files = sorted(out_files, key=lambda x: os.path.getctime(os.path.join(FOLDER_OUTPUT, x)), reverse=True)

out_files

['comparators_107.pkl',
 'comparators_1684.pkl',
 'comparators_1912.pkl',
 'comparators_3437.pkl',
 'comparators_0.pkl',
 'comparators_348.pkl',
 'comparators_686.pkl',
 'comparators_414.pkl',
 'comparators_698.pkl',
 'comparators_3980.pkl']

In [8]:
import pickle
#pick first id from out_files
id = out_files[0].split('_')[1]

with open(f'{FOLDER_OUTPUT}/comparators_{id}', 'rb') as f:
    comparators = pickle.load(f)


In [9]:
import pickle
# with open(f'{FOLDER_OUTPUT}/fitted_graphs_698.pkl', 'rb') as f:
#     fitted_graphs = pickle.load(f)
# with open(f'{FOLDER_OUTPUT}/summary_dfs_698.pkl', 'rb') as f:
#     summary_dfs = pickle.load(f)

In [10]:
len(comparators)

10

In [11]:
dfs = []
for i in range(len(comparators)):
    dfs.append(comparators[i].summary_df)

dfs[0]

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,3980.edges,Original,NaN,N/A,True,52,146,0.110106,0.461727,2.566596,5,0.160451,4,44
1,3980.edges,LG,0.596708,"d=1, sigma=-4.3053",True,52,158,0.119155,0.091811,2.321267,4,0.047564,1,52
2,3980.edges,WS,0.459965,20.115556,True,52,156,0.117647,0.113629,2.355204,4,-0.028544,1,52
3,3980.edges,BA,0.560231,3.666667,True,52,147,0.110860,0.191458,2.331825,4,-0.130909,1,52
4,3980.edges,ER,0.658101,0.157778,True,52,195,0.147059,0.127510,2.147059,4,-0.026210,1,52
5,3980.edges,GRG,13.313399,1.0,True,52,1261,0.950980,0.956873,1.049020,2,-0.060541,1,52


In [12]:
pd.concat(dfs)

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,3980.edges,Original,NaN,N/A,True,52,146,0.110106,0.461727,2.566596,5,0.160451,4,44
1,3980.edges,LG,0.596708,"d=1, sigma=-4.3053",True,52,158,0.119155,0.091811,2.321267,4,0.047564,1,52
2,3980.edges,WS,0.459965,20.115556,True,52,156,0.117647,0.113629,2.355204,4,-0.028544,1,52
3,3980.edges,BA,0.560231,3.666667,True,52,147,0.110860,0.191458,2.331825,4,-0.130909,1,52
4,3980.edges,ER,0.658101,0.157778,True,52,195,0.147059,0.127510,2.147059,4,-0.026210,1,52
5,3980.edges,GRG,13.313399,1.0,True,52,1261,0.950980,0.956873,1.049020,2,-0.060541,1,52
0,698.edges,Original,NaN,N/A,True,61,270,0.147541,0.733202,1.944872,4,0.287962,3,40
1,698.edges,LG,0.526334,"d=2, sigma=-3.7050",True,61,303,0.165574,0.188064,2.001639,3,-0.035650,1,61
2,698.edges,BA,0.547897,5.0,True,61,280,0.153005,0.266403,2.028962,3,-0.136143,1,61
3,698.edges,WS,0.961434,20.178889,True,61,244,0.133333,0.099150,2.151366,4,-0.094036,1,61


In [13]:
# Create a dictionary to store rankings for each graph
rankings = {}

# Go through each df in dfs
for df in dfs:
    graph_name = df['graph_filename'].iloc[0]  # Get graph filename
    
    # Sort models by GIC value (excluding 'Original' which has NaN GIC)
    model_ranks = df[df['model'] != 'Original'].sort_values('gic_value')['model'].tolist()
    
    # Create rankings dictionary for this graph (1 is best since lowest GIC is best)
    rankings[graph_name] = {
        model: rank+1 for rank, model in enumerate(model_ranks)
    }
    
    # Add nodes and edges from Original model
    original_row = df[df['model'] == 'Original'].iloc[0]
    rankings[graph_name]['nodes'] = original_row['nodes']
    rankings[graph_name]['edges'] = original_row['edges']

# Get unique models (excluding 'Original')
models = sorted(list(set([model for df in dfs for model in df['model'].unique() if model != 'Original'])))

# Create DataFrame with rankings
ranking_df = pd.DataFrame(rankings).T

# Ensure all models are present as columns
for model in models:
    if model not in ranking_df.columns:
        ranking_df[model] = np.nan

# Keep only the model columns in a consistent order, plus nodes and edges
ranking_df = ranking_df[models + ['nodes', 'edges']]

print("Rankings by GIC value (lower is better):")
display(ranking_df)

Rankings by GIC value (lower is better):


,BA,ER,GRG,LG,WS,nodes,edges
3980.edges,2,4,5,3,1,52,146
698.edges,2,4,5,1,3,61,270
414.edges,1,2,5,3,4,150,1693
686.edges,1,4,5,2,3,168,1656
348.edges,2,3,5,4,1,224,3192
0.edges,3,4,5,2,1,333,2519
3437.edges,1,4,5,3,2,534,4813
1912.edges,3,1,5,2,4,747,30025
1684.edges,1,3,5,2,4,786,14024
107.edges,1,2,5,3,4,1034,26749


In [14]:
ranking_df.mean(axis=0)

BA          1.7
ER          3.1
GRG         5.0
LG          2.5
WS          2.7
nodes     408.9
edges    8508.7
dtype: float64

In [15]:
ranking_df['nodes'].describe()

count      10.000000
mean      408.900000
std       345.822514
min        52.000000
25%       154.500000
50%       278.500000
75%       693.750000
max      1034.000000
Name: nodes, dtype: float64